<img src="resources/tag_metters_Logo.png" width="299" height="45">

## #Tag is invisible but it does actually track the trends
To understand how tags play the role by using it on NYTimes daily and how people really react about the topic by looking at other media platforms through NYTimes tags.

In [4]:
import sys
sys.executable

'/usr/local/opt/python/bin/python3.7'

In [5]:
sys.path

['/Users/hh/Documents/Pratt/Adv.ProjectsinVis/Monthly-Frequency-of-NYTimes-Tag',
 '/usr/local/Cellar/python/3.7.7/Frameworks/Python.framework/Versions/3.7/lib/python37.zip',
 '/usr/local/Cellar/python/3.7.7/Frameworks/Python.framework/Versions/3.7/lib/python3.7',
 '/usr/local/Cellar/python/3.7.7/Frameworks/Python.framework/Versions/3.7/lib/python3.7/lib-dynload',
 '',
 '/Users/hh/Library/Python/3.7/lib/python/site-packages',
 '/usr/local/lib/python3.7/site-packages',
 '/usr/local/lib/python3.7/site-packages/IPython/extensions',
 '/Users/hh/.ipython']

In [216]:
import json
import requests
import pandas as pd
import numpy as np
import operator
import time
import praw
import nltk
import configparser

from apiclient import discovery
from datetime import datetime
from dateutil.relativedelta import relativedelta

from pytrends.request import TrendReq
from praw.models import MoreComments
from googleapiclient import discovery
from textblob import TextBlob
from pandas.io.json import json_normalize

import pprint
import matplotlib.pyplot as plt
import seaborn as sns

In [217]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey

from sqlalchemy.orm import sessionmaker

In [218]:
# Use CofigParser to safely store the password or key
config = configparser.ConfigParser()
config.read('key_pair.ini')

Times_key = config['Times']['key']

Reddit_client_id = config['Reddit']['client_id']
Reddit_client_secret = config['Reddit']['client_secret']
Reddit_username = config['Reddit']['username']
Reddit_password = config['Reddit']['password']
Reddit_user_agent = config['Reddit']['user_agent']

Youtube_service_name = config['Youtube']['YOUTUBE_API_SERVICE_NAME']
Youtube_API_version = config['Youtube']['YOUTUBE_API_VERSION']
Youtube_developer_key = config['Youtube']['DEVELOPER_KEY']

### Storing Data using SQLalchemy

In [6]:
Base = declarative_base()

class Init(Base): 
    __tablename__ = 'metadata'
    __table_args__ = {'extend_existing': True}

    id = Column(Integer, primary_key=True)
    Tag = Column(String(255))
    Frequency = Column(String(255))
    Title = Column(String(255))
    Date = Column(String(25))
    Url = Column(String(255))
    img_URL = Column(String(255))

### New York Times API
https://developer.nytimes.com/apis

Archive API
- Finds all articles's metadata from archive APi (about 7000 every month): title, section_name, period, url, word_count, keywords

In [212]:
def get_NYTimes_metadata():

    # To get most frequently used tag in NYTimes
    today = str(datetime.today())
    monthly_archive = {}
    frequent_tags_archive = {}

    for yy in reversed(range(2019, int(today[:4]) + 1)):
        if str(yy) == today[:4]:
            ends = int(today[5:7]) + 1
            start = 1
        else:
            ends = 13
            if '0' in today[5:7]:
                start = int(today[6:7])
            else:
                start = int(today[5:7])

        for mm in reversed(range(start,ends)):
            if len(str(mm)) == 1:
                date_not_duplicate = str(yy) + '-0' + str(mm)
            else:
                date_not_duplicate = str(yy) + '-' + str(mm)
#             print('--match--',date_not_duplicate)
#             print('in it?', monthly_archive.keys())
            if date_not_duplicate not in monthly_archive:
                parameters = {'api-key': Times_key}           
                archived_Url = 'https://api.nytimes.com/svc/archive/v1/'+ str(yy) +'/'+ str(mm) +'.json'
                archives = requests.get(archived_Url, params=parameters).json()

                monthly_article = []
                for a in archives['response']['docs']:
                    articles = {}
                    articles['title'] = a['headline']['main']
                    articles['pub_date'] = a['pub_date'][:10]
                    articles['url'] = a['web_url']
                    if len(a['multimedia']) !=0 and a['multimedia'][0]['url']:
                        articles['thm_img'] = 'https://static01.nyt.com/' + a['multimedia'][0]['url']
                    else:
                        articles['thm_img'] = 'no_image_found'
                    articles['tags'] = [''.join(tag['value']) for tag in a['keywords']]
                    monthly_article.append(articles)
                if len(str(mm)) == 1:
                    mm = '0' + str(mm)
#                 print('', str(yy)+'-'+str(mm))
                monthly_archive[str(yy)+'-'+str(mm)] = monthly_article

                tag_arr = []
                for m in monthly_article:
                    for t in m['tags']:
                        for string in t.split(', '):
                            tag_arr.append(', '.join(string.split(', ')))

                count_tag = {}
                for tag in tag_arr:
                    if tag in ['Trump', 'Donald J']:
                        tag = 'Donald Trump'
                    if tag in ['Joseph R Jr', 'Biden']:
                        tag = 'Joe Biden'
                    if tag in ['Brett M', 'Supreme Court (US)', 'Kavanaugh']:
                        tag = 'Brett Kavanaugh'
                    if tag in ['Putin', 'Vladimir V']:
                        tag = 'Putin'
                    if tag in ['Fla', 'Parkland']:
                        tag = 'Parkland'
                    if tag in ['Coronavirus Aid', 'Relief', 'and Economic Security Act (2020)']:
                        tag = 'Coronavirus Aid, Relief, and Economic Security Act (2020)'
                    if tag in ['School Shootings and Armed Attacks']:
                        tag = 'School Shootings'
                    if tag in ['Shutdowns (Institutional)']:
                        tag = 'Shutdowns'
                    if tag in ['New York City', 'NYC','NY)','New York State' ]:
                        tag = 'New York State'
                    if tag in ['States (US)', 'United States']:
                         tag = 'United States'
                    if 'Russian Interference in 2016' in tag:
                        tag = 'Russian interference in the 2016 United States elections'
                    if tag in ['Homosexuality and Bisexuality']:
                        tag = 'Homosexuality'
                    if 'Trump-Ukraine' in tag:
                        tag = 'Trump-Ukraine'
                    if tag in ['Biden, Joseph R Jr', 'Biden']:
                        tag = 'Joe Biden'
                    if tag in ['Sanders, Bernard', 'Sanders', 'Bernard', 'Bernie Sanders']:
                        tag = 'Bernie Sanders'
                    if tag in ['Pete Buttigieg', 'Buttigieg, Pete (1982- )', 'Buttigieg', 'Pete Buttigieg', 'Pete (1982- )']:
                        tag = 'Pete Buttigieg'
                    if 'Quarantine' in tag:
                        tag = 'Quarantines'
                    if 'Deaths' in tag:
                        tag = 'Deaths'
                    if 'Child' in tag:
                        tag = 'Parenting'
                    if tag in ['Blacks', 'Black People']:
                        tag = 'Blacks'
                    if tag in ['Donald Trump', 'Weddings and Engagements', 'Women and Girls', 'New York State', 'Democratic Party', 'Primaries and Caucuses', 'United States Politics and Government', 'Politics and Government', 'Books and Literature', 'Television', 'Movies', 'Real Estate and Housing (Residential)', 'United States', 'United States International Relations' 'Primaries and Caucuses', 'United States Economy', 'Elections']:
                        tag = ''

                    if tag is not '':
                        if tag in count_tag:
                            count_tag[tag] += 1
                        else:
                            count_tag[tag] = 1   

                # This variable is what we want to get in NYTimes
                tags_with_frequency = sorted(count_tag.items(),key=operator.itemgetter(1),reverse=True)[:11]
                frequent_tags_archive[str(yy)+'-'+str(mm)] = tags_with_frequency

    return monthly_archive, frequent_tags_archive

In [ ]:
monthly_archive = get_NYTimes_metadata()[0]
frequent_tags_archive = get_NYTimes_metadata()[1]
frequent_tags_archive

### Find the most appeared tags among these days

In [227]:
frequent_tags_archive

{'2020-05': [('Coronavirus (2019-nCoV)', 990),
  ('Quarantines', 346),
  ('Coronavirus Aid, Relief, and Economic Security Act (2020)', 204),
  ('Parenting', 165),
  ('Deaths', 161),
  ('Shutdowns', 108),
  ('Joe Biden', 98),
  ('Epidemics', 87),
  ('Presidential Election of 2020', 73),
  ('Labor and Jobs', 58),
  ('Unemployment', 53)],
 '2020-04': [('Coronavirus (2019-nCoV)', 3978),
  ('Parenting', 1278),
  ('Quarantines', 983),
  ('Coronavirus Aid, Relief, and Economic Security Act (2020)', 807),
  ('Deaths', 568),
  ('Presidential Election of 2020', 537),
  ('Epidemics', 435),
  ('Joe Biden', 370),
  ('Shutdowns', 264),
  ('Hospitals', 249),
  ('Babies and Infants', 206)],
 '2020-03': [('Coronavirus (2019-nCoV)', 3463),
  ('Quarantines', 860),
  ('Joe Biden', 817),
  ('Presidential Election of 2020', 813),
  ('Bernie Sanders', 705),
  ('Epidemics', 650),
  ('Shutdowns', 372),
  ('Deaths', 333),
  ('Parenting', 310),
  ('California', 234),
  ('Hospitals', 206)],
 '2020-02': [('Preside

In [228]:
dicts= {}
for periode in frequent_tags_archive:
    count = 0
    for tags in frequent_tags_archive[periode]:
        if tags[0] not in dicts:
#             count = count + 1
            dicts[tags[0]] = tags[1]
        else:
            dicts[tags[0]] = dicts[tags[0]] + tags[1]
print(dicts)

{'Coronavirus (2019-nCoV)': 9031, 'Quarantines': 2189, 'Coronavirus Aid, Relief, and Economic Security Act (2020)': 1011, 'Parenting': 3867, 'Deaths': 3163, 'Shutdowns': 744, 'Joe Biden': 4579, 'Epidemics': 1441, 'Presidential Election of 2020': 6000, 'Labor and Jobs': 58, 'Unemployment': 53, 'Hospitals': 455, 'Babies and Infants': 206, 'Bernie Sanders': 2395, 'California': 234, 'Pete Buttigieg': 559, 'China': 1073, 'Michael R': 259, 'Bloomberg': 242, 'Trump-Ukraine': 2111, 'United States International Relations': 2327, 'Impeachment': 1352, 'Senate': 377, 'Iran': 345, 'United States Defense and Military Forces': 268, 'Suleimani': 257, 'Qassim': 257, 'House of Representatives': 601, 'Republican Party': 348, 'Demonstrations': 497, 'Computers and the Internet': 194, 'Ukraine': 568, 'Art': 579, 'Syria': 227, 'Fashion and Apparel': 171, 'Shooting (2019)': 249, 'International Trade and World Market': 414, 'Immigration and Emigration': 679, 'Tex': 168, 'El Paso': 166, 'Protests and Riots': 36

In [9]:
general_tags = [('Donald Trump', 28),
 ('New York State', 28),
 ('United States Politics and Government', 28),
 ('Politics and Government', 28),
 ('Books and Literature', 28),
 ('Television', 28),
 ('Movies', 28),
 ('Real Estate and Housing (Residential)', 28),
 ('United States', 27),
 ('United States International Relations', 26),
 ('Primaries and Caucuses', ),
 ('United States Economy', ),
 ('United States Defense and Military Forces', ),
 ('Appointments and Executive Changes',),
 ('Weddings and Engagements', ),
 ('Elections',)]#,
#  ('Democratic Party', 25),
#  ('China', 25),
#  ('Art', 25),
#  ('Deaths (Obituaries)', 24),
#  ('Women and Girls', 21),
#  ('Elections', 20),
#  ('Republican Party', 20),
#  ('House of Representatives', 18),
#  ('Theater', 17),
#  ('Presidential Election of 2020', 16),
#  ('Weddings and Engagements', 15)]

### Pytrends API
https://pypi.org/project/pytrends/

Sample codes to see Google doesn't block my IP

In [219]:
pytrends = TrendReq(hl='en-US', tz=360, timeout=(10,25))
tag_arr = ['Coronavirus (2019-nCoV)']
pytrends.build_payload(tag_arr, cat=0, timeframe='2020-04-01 ' + str(datetime.now())[:10], geo='', gprop='')
time.sleep(2)
df = pytrends.interest_over_time().reset_index()
df.head()

,date,Coronavirus (2019-nCoV),isPartial
0,2020-04-01,75,False
1,2020-04-02,44,False
2,2020-04-03,56,False
3,2020-04-04,74,False
4,2020-04-05,87,False


In [114]:
df['date'].iloc[0]

Timestamp('2020-04-01 00:00:00')

In [117]:
today = str(datetime.today())
print('2019-05-01 '+ today[:10])

2019-05-01 2020-05-11


In [118]:
# Get a unique tag collection for the search query
def get_trends_Tags(frequent_tags_archive): 
    today = str(datetime.today())
    frequent_tag_only = {}
    
    for time_period in frequent_tags_archive:
        tag_only = []
        for each in frequent_tags_archive[time_period]:    
            tag_only.append(each[0])
            frequent_tag_only[time_period] = tag_only
        
    pytrends = TrendReq(hl='en-US', tz=360)
    monthly_interests = {}
    
    for period in frequent_tag_only:
        data = []
        for tag in frequent_tag_only[period]:
            tag_arr = []
            tag_arr.append(tag)
            print(tag)
            interest_over_time = {}
            
            try:
                pytrends.build_payload(tag_arr, cat=0, timeframe = '2019-05-01 '+ today[:10], geo='', gprop='')
                df = pytrends.interest_over_time().reset_index()
            except Exception as e:
                print(tag_arr, ', and which reason? ', e)
                pass
            
            interest_over_time['Tag'] = tag
            try:
                interest_over_time['StartDate'] = one_month_ago
                interest_over_time['EndDate'] = one_month_later
                interest_over_time['Rate'] = [rate for rate in df[tag]]
            except Exception as e:
                pass
            data.append(interest_over_time)     
        monthly_interests[period] = data
    return monthly_interests

In [119]:
monthly_interests = get_trends_Tags(frequent_tags_archive)
monthly_interests

Coronavirus (2019-nCoV)
Quarantines
Deaths
Coronavirus Aid, Relief, and Economic Security Act (2020)


KeyboardInterrupt: 

### Reddit API
https://praw.readthedocs.io/en/latest/

In [222]:
def unique_top_tag_only(frequent_tags_archive):
    unique_top_tag_only={}
    for periode in frequent_tags_archive:
        for tag_with_F in frequent_tags_archive[periode]:
            if tag_with_F[0] in unique_top_tag_only:
                unique_top_tag_only[tag_with_F[0]] = tag_with_F[1] + unique_top_tag_only[tag_with_F[0]]
            else:
                unique_top_tag_only[tag_with_F[0]] = tag_with_F[1]

    unique_top_tag_only = sorted(unique_top_tag_only.items(),key=operator.itemgetter(1),reverse=True)[:30]

    return unique_top_tag_only       

In [223]:
unique_top_tag_only = unique_top_tag_only(frequent_tags_archive)
unique_top_tag_only

[('Coronavirus (2019-nCoV)', 9031),
 ('Presidential Election of 2020', 6000),
 ('Joe Biden', 4579),
 ('Parenting', 3867),
 ('Deaths', 3163),
 ('Bernie Sanders', 2395),
 ('United States International Relations', 2327),
 ('Quarantines', 2189),
 ('Trump-Ukraine', 2111),
 ('Epidemics', 1441),
 ('Impeachment', 1352),
 ('China', 1073),
 ('Coronavirus Aid, Relief, and Economic Security Act (2020)', 1011),
 ('Shutdowns', 744),
 ('Immigration and Emigration', 679),
 ('House of Representatives', 601),
 ('Art', 579),
 ('Ukraine', 568),
 ('Pete Buttigieg', 559),
 ('Demonstrations', 497),
 ('Hospitals', 455),
 ('International Trade and World Market', 414),
 ('Senate', 377),
 ('Protests and Riots', 366),
 ('Republican Party', 348),
 ('Theater', 348),
 ('Iran', 345),
 ('United States Defense and Military Forces', 268),
 ('Michael R', 259),
 ('Suleimani', 257)]

In [233]:
def get_reddit_comments(unique_top_tag_only):
    
    reddit = praw.Reddit(client_id = Reddit_client_id,
                         client_secret = Reddit_client_secret,
                         username = Reddit_username,
                         password = Reddit_password,
                         user_agent = Reddit_user_agent)
#     times_metadata = {'2020-04':['Coronavirus (2019-nCoV)', 'Quarantines', 'Parenting'], '2019-04': ['China', 'Epidemics', 'Deaths (Fatalities)']}

    reddit_metadata = []
    for tag_frequency in unique_top_tag_only[:5]:
        subreddit = reddit.subreddit('all')
        time.sleep(2)

        each_tag = {}
        for i, post in enumerate(subreddit.search(tag_frequency[0], sort='relevance', syntax='lucene', limit=3)):
            count = 0;
            if not post.stickied:
                each_tag['Tag'] = tag_frequency[0]
                print('* *',tag_frequency[0])
#                 each_tag['Title'] = post.title
#                 each_tag['CreatedAt'] = datetime.fromtimestamp(post.created_utc).isoformat()[:10]
#                 each_tag['DiscussionAbout'] = post.url
                print('* * *', post.url)
                each_tag_comments = post.comments.list()
                comments_arr = []
                for comment in each_tag_comments:
                    if isinstance(comment, MoreComments):
                        continue
                    comments_arr.append(comment.body)
                    count = count + len(comments_arr)
                each_tag['comments'] = count

        reddit_metadata.append(each_tag)
                    
    return reddit_metadata

In [234]:
get_reddit_comments(unique_top_tag_only)

* * Coronavirus (2019-nCoV)
* * * https://www.reddit.com/r/askscience/comments/ewwmem/have_a_question_about_the_2019_novel_coronavirus/
* * Coronavirus (2019-nCoV)
* * * https://twitter.com/WHO/status/1217043229427761152
* * Coronavirus (2019-nCoV)
* * * https://www.medrxiv.org/content/10.1101/2020.02.07.20021154v1.full.pdf
* * Presidential Election of 2020
* * * https://www.reddit.com/r/SampleSize/comments/geuqnv/casual_who_would_you_like_to_see_win_the_us/
* * Presidential Election of 2020
* * * https://www.reddit.com/r/AskOuija/comments/f1iykn/will_win_the_presidential_election_of_2020/
* * Presidential Election of 2020
* * * https://www.reddit.com/r/AskOuija/comments/9peqyy/and_our_final_candidate_for_the_presidential/
* * Joe Biden
* * * https://www.reddit.com/r/politics/comments/g0orjb/megathread_senator_bernie_sanders_endorses_joe/
* * Joe Biden
* * * https://www.reddit.com/r/politics/comments/g19ge3/megathread_barack_obama_endorses_joe_biden_for/
* * Joe Biden
* * * https://www

[{'Tag': 'Coronavirus (2019-nCoV)', 'comments': 40755},
 {'Tag': 'Presidential Election of 2020', 'comments': 105},
 {'Tag': 'Joe Biden', 'comments': 122265},
 {'Tag': 'Parenting', 'comments': 122265},
 {'Tag': 'Deaths', 'comments': 124750}]

## Not using anymore

- Top Stories API: (about 30 articles by 7 days): title, pub_date, url, section, des_facet, geo_facet
- Newswire API: Finds by Archive API's url : des_facet
- Most Popular API
- Top Stories API, Times Tags API, Community API

### Youtube API
https://developers.google.com/youtube/v3/docs/search/list?hl=en_US

In [235]:
# creating Youtube Resource Object 
youtube_object = discovery.build(Youtube_service_name, Youtube_API_version, developerKey = Youtube_developer_key)

In [238]:
def get_youtube_comments(unique_top_tag_only):  
    # calling the search.list method to retrieve youtube search results 
    
    max_results = 10
    youtube_metadata = []
    for tag_frequency in unique_top_tag_only[:2]:
        viewCount = 0
        commentCount = 0
        likeCount = 0
        video = {}
        search_tags = youtube_object.search().list(q = tag_frequency[0], part = "id, snippet", order = 'relevance', maxResults = max_results, publishedAfter = "2019-05-01T00:00:00Z").execute() 
        print('start: ', tag_frequency[0])
        for item in search_tags.get("items", []):
            time.sleep(2)
            video["Tag"] = tag_frequency[0]
#             video["videoId"] = item["id"]["videoId"]
#             video["publishedAt"] = item['snippet']['publishedAt']
#             video["title"] = item['snippet']['title']
#             video['description'] = item['snippet']['description']
            stats = youtube_object.videos().list(part='statistics, snippet', id=item["id"]["videoId"]).execute()
            new_viewCount = int(stats.get("items", [])[0]['statistics']['viewCount'])
            new_commentCount = int(stats.get("items", [])[0]['statistics']['commentCount'])
            new_likeCount =  int(stats.get("items", [])[0]['statistics']['likeCount'])
            viewCount = viewCount + new_viewCount
            commentCount = commentCount + new_commentCount
            likeCount = likeCount + new_likeCount
            video['viewCount'] = viewCount
            video['commentCount'] = commentCount
            video['likeCount'] = likeCount
        youtube_metadata.append(video)
    return youtube_metadata

In [239]:
get_youtube_comments(unique_top_tag_only)

start:  Coronavirus (2019-nCoV)


KeyError: 'commentCount'

## Reddit

In [ ]:
nltk.download('averaged_perceptron_tagger')

In [ ]:
def get_sentimental_chart(reddit_metadata):
    tags_metadata = []
    for single_tag in reddit_metadata:
        tag_sentiment = {}
        avg_polarity = 0
        pos_count = 0
        neg_count = 0
        neutral_count = 0
        for single_comment in single_tag['Comments']: 
            
            # Get Word Sentimental analysis
            blobed_word = TextBlob(single_comment)
            if blobed_word.sentiment.polarity > 0:
                pos_count += 1
            elif blobed_word.sentiment.polarity == 0:
                neutral_count += 1
            else:
                neg_count += 1      
            avg_polarity += blobed_word.sentiment.polarity
            
            tag_sentiment['tag'] = single_tag['Tag']    
            tag_sentiment['avg_polarity'] = avg_polarity / len(single_tag['Comments'])
            tag_sentiment['pos_count'] = pos_count
            tag_sentiment['neg_count'] = neg_count
            tag_sentiment['neutral_count'] = neutral_count
            
            # Get Adjective words' frequencies
            adgs_arr = []
            for blobed_tags in blobed_word.tags:
                if blobed_tags[1] in ['JJ', 'JJR', 'JJS']:
                    adgs_arr.append(blobed_tags[0])
            adg_frequency = {}
            for adg in adgs_arr: 
                if adg in adg_frequency:
                    adg_frequency[adg] += 1
                else:
                    adg_frequency[adg] = 1 
            tag_sentiment['adg_frequency'] = sorted(adg_frequency.items(),key=operator.itemgetter(1),reverse=True)[:3]       
        tags_metadata.append(tag_sentiment)
        
    return tags_metadata

In [138]:
frequent_tags_archive = {'2020-04': [('Coronavirus (2019-nCoV)', 3591),
  ('Quarantines', 822)]}

In [173]:
def get_reddit_Posts(times_metadata):   
#     result = {}
#     for time_period in times_metadata:
#         data = []
#         for each in times_metadata[time_period]:     
#             data.append(each['Tag'])
#             result[time_period] = data  
            
    reddit = praw.Reddit(client_id = Reddit_client_id,
                         client_secret = Reddit_client_secret,
                         username = Reddit_username,
                         password = Reddit_password,
                         user_agent = Reddit_user_agent)
    result = {'2020-04':['Coronavirus (2019-nCoV)', 'Quarantines', 'Parenting'], '2019-04': ['China', 'Epidemics', 'Deaths (Fatalities)']}
 
    reddit_metadata = []
    for period in result:
        print(period + ': ', len(result[period]))
        reddit_posts = {}
        add_post_arr = []
        for tag in result[period]:
            subreddit = reddit.subreddit('all')
            time.sleep(2)
            for i, post in enumerate(subreddit.search(tag, sort='relevance', syntax='lucene', limit=100)):
                if not post.stickied:
                    each_tag = {}
                    each_tag['Tag'] = tag
#                     print('--1--',tag)
                    each_tag['Title'] = post.title
                    each_tag['CreatedAt'] = datetime.fromtimestamp(post.created_utc).isoformat()[:10]
                    each_tag['DiscussionAbout'] = post.url
                    each_tag['Ups'] = post.ups
                    each_tag['Downs'] = post.downs
                    add_post_arr.append(each_tag)
        reddit_posts[period] = add_post_arr
        reddit_metadata.append(reddit_posts)
#         print('--2--',reddit_metadata)
    
#     add_tag = {}
#     for item in reddit_metadata:
#         for period in item: 
#             monthly_posts = []
#             byTime = {}
            
#             for each_post in item[period]:
#                 each_post['CreatedAt'] = each_post['CreatedAt'][:7]

#                 if each_post['Tag'] in add_tag:
#                     if each_post['CreatedAt'] in byTime:
#                         byTime[each_post['CreatedAt']][0] += 1
#                         byTime[each_post['CreatedAt']][1] += each_post['Ups']
#                     else:
#                         add_freq_votes = []
#                         byTime[each_post['CreatedAt']] = add_freq_votes
#                         add_freq_votes.append(1)
#                         add_freq_votes.append(each_post['Ups'])
#                 else:
#                     add_tag[each_post['Tag']] = byTime
#                     if each_post['CreatedAt'] in byTime:
#                         byTime[each_post['CreatedAt']][0] += 1
#                         byTime[each_post['CreatedAt']][1] += each_post['Ups']
#                     else:
#                         add_freq_votes = []
#                         byTime[each_post['CreatedAt']] = add_freq_votes
#                         add_freq_votes.append(1)
#                         add_freq_votes.append(each_post['Ups'])
# #                     print('--1--', add_tag)
#             monthly_posts.append(add_tag)
# #             print('--2--', monthly_posts)
    
# #     print('--3--', monthly_posts)
    return reddit_metadata